# Steps to getting an Authorization Token
1. Register a web application on MyAnimeList as described in https://myanimelist.net/blog.php?eid=835707
2. Store your MyAnimeList Client ID and Client Secret in the files `data/mal_authentication/clientid.txt` and `data/mal_authentication/clientsecret.txt`
3. run the notebook

In [1]:
import json
import os
import secrets

import requests

os.chdir("../../data/mal_authentication")
with open("clientid.txt") as f:
    CLIENT_ID = f.read().strip()
with open("clientsecret.txt") as f:
    CLIENT_SECRET = f.read().strip()

# 1. Generate a new Code Verifier / Code Challenge.
def get_new_code_verifier() -> str:
    token = secrets.token_urlsafe(100)
    return token[:128]


# 2. Print the URL needed to authorise your application.
def print_new_authorisation_url(code_challenge: str):
    global CLIENT_ID

    url = f"https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&code_challenge={code_challenge}"
    print(f"Authorise your application by clicking here: {url}\n")
    print(
        f"After pressing 'Allow', it's okay if the page fails to open, you just"
        " need to copy the URL that it tries to redirect you to\n"
    )


# 3. Once you've authorised your application, you will be redirected to the webpage you've
#    specified in the API panel. The URL will contain a parameter named "code" (the Authorisation
#    Code). You need to feed that code to the application.
def generate_new_token(authorisation_code: str, code_verifier: str) -> dict:
    global CLIENT_ID, CLIENT_SECRET

    url = "https://myanimelist.net/v1/oauth2/token"
    data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": authorisation_code,
        "code_verifier": code_verifier,
        "grant_type": "authorization_code",
    }

    response = requests.post(url, data)
    response.raise_for_status()  # Check whether the requests contains errors

    token = response.json()
    response.close()
    print("Token generated successfully!")

    with open("token.json", "w") as file:
        json.dump(token, file, indent=4)
        print('Token saved in "token.json"')

    return token


# 4. Test the API by requesting your profile information
def print_user_info(access_token: str):
    url = "https://api.myanimelist.net/v2/users/@me"
    response = requests.get(url, headers={"Authorization": f"Bearer {access_token}"})

    response.raise_for_status()
    user = response.json()
    response.close()

    print(f"\n>>> Greetings {user['name']}! <<<")

In [2]:
code_verifier = code_challenge = get_new_code_verifier()
print_new_authorisation_url(code_challenge)

Authorise your application by clicking here: https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=2c833385ced9d77b887078684b405fbd&code_challenge=bP99zJBM8S45vHXwouctDZ4anip87E6pCXnikSe2scr3-Doy99p-VGSmA415VNgXOT-KvxkAK5npajmCie_BntupD_fnGN_MFD-sNLfVgBYkOZKuSXl_UiQX6y_PMRO0

After pressing 'Allow', it's okay if the page fails to open, you just need to copy the URL that it tries to redirect you to



In [3]:
authorisation_code = input("Copy-paste the Authorisation Code or URL: ").strip()
default_url_prefix = "http://localhost/oauth?code="
if authorisation_code.startswith(default_url_prefix):
    authorisation_code = authorisation_code[len(default_url_prefix) :]
token = generate_new_token(authorisation_code, code_verifier)

Copy-paste the Authorisation Code or URL:  http://localhost/oauth?code=def502008182218f5fa0481eb2c4861bc2e7f053ea4b01c2bd9495080c36f777c9bfcd4fd0d305a43177e8062831a4f28bb5480f73cbfd52da954f8693f28ce1e21c7ac8f6b309f24be65feeca522cfb56418e18023880122cbd07b6df1b11cefd364693ac788243c8f1204764270d62fba8372898a853c8d7204b9558ca3eccfdc62aef2aea1e2e73c7a1aec0f82c96beaaf93368a16b43fa95d31e90778fa5da3cbd56bf9e6e60a79bcf6c0028c1034368dd3af6c533098277221811564aae8e7e04508c695d1264bda3578c46fb95867f88828d39d4cd91bc2a4138814a3ff1926df23fc55117e1043b464421f1897b507e132c9afe2b34395d901bc4e25a9bc81349cd3c9118727ccd9e3e74fd3a1fe4264430727ca41bd9884b428193e45fe00954e72600442370467a489eacffddd6f69d75c7b44d5cbacf69b746fb30ceff0814b53612038f60f90c01d08a0b1e6cc622166c4685ed50cba31bef110128a7201c9b899401a55c1c5ae36c066dbac40b455e0e3933ddc4cd9054567c8ab231c9b0c4b1c4a743966c14dd1d80effaa6ddaa7498e47ee7622b2edde7f8e61206cda8b980e1a434c4f8baa9215d5f2c8aceedfbf7603ea42aed532a5122170548fa00280151c06f4f51bcd7cae89e7

Token generated successfully!
Token saved in "token.json"


In [6]:
print_user_info(token['access_token'])


>>> Greetings Fro116! <<<
